In [ ]:
# Databricks notebook source

# Exercice 02 : Transformations et Agregations## Objectifs pedagogiquesA la fin de cet exercice, vous serez capable de :- Utiliser les fonctions d'agregation (sum, avg, count, min, max)- Regrouper des donnees avec groupBy- Calculer des statistiques par groupe- Utiliser les fonctions de date et heure- Faire des jointures entre tables- Creer des pivots## Duree estimee : 60 minutes

## PARTIE 1 : Concepts theoriques### Fonctions d'agregationLes fonctions d'agregation calculent une valeur unique a partir de plusieurs lignes :- `count()` : Nombre de lignes- `sum()` : Somme- `avg()` : Moyenne- `min()` : Minimum- `max()` : Maximum- `first()` : Premiere valeur- `last()` : Derniere valeur- `collect_list()` : Liste de toutes les valeurs- `collect_set()` : Ensemble de valeurs uniques### GroupBy`groupBy()` permet de regrouper les lignes selon une ou plusieurs colonnes,puis d'appliquer des agregations sur chaque groupe.

## PARTIE 2 : Agregations simples

In [ ]:

print("EXERCICE 2.1 : Agregations globales")
print("=" * 70)

from pyspark.sql.functions import count, countDistinct, avg, min, max

# Charger les donnees
df_gares = spark.table("gares_silver")

# Agregations simples
print("Statistiques globales :")
df_stats = df_gares.agg(
    count("*").alias("nombre_total_gares"),
    countDistinct("code_departement").alias("nombre_departements"),
    countDistinct("segment_clean").alias("nombre_segments"),
    avg("latitude").alias("latitude_moyenne"),
    min("latitude").alias("latitude_min"),
    max("latitude").alias("latitude_max")
)

df_stats.show()


## EXERCICE PRATIQUE 2.2Calculez les statistiques suivantes :- Nombre total de gares- Nombre de trigrammes uniques- Longitude minimale et maximale- Nombre de categories de gares distinctes

In [ ]:

# A COMPLETER
from pyspark.sql.functions import count, countDistinct, min, max

df_stats_exercice = df_gares.agg(
    count("*").alias("total_gares"),
    countDistinct("trigramme").alias("trigrammes_uniques"),
    min("longitude").alias("longitude_min"),
    max("longitude").alias("longitude_max"),
    countDistinct("categorie_gare").alias("nombre_categories")
)

df_stats_exercice.show()


## PARTIE 3 : GroupBy et agregations

In [ ]:

print("EXERCICE 2.3 : Regroupement par segment")
print("=" * 70)

from pyspark.sql.functions import count, avg, col

# Regrouper par segment et compter
df_par_segment = df_gares.groupBy("segment_clean").agg(
    count("*").alias("nombre_gares"),
    avg("latitude").alias("latitude_moyenne")
).orderBy("segment_clean")

print("Repartition par segment :")
df_par_segment.show()

# Regrouper par departement
df_par_dept = df_gares.groupBy("code_departement").agg(
    count("*").alias("nombre_gares")
).orderBy(col("nombre_gares").desc())

print("\nTop 10 departements avec le plus de gares :")
df_par_dept.show(10)


## EXERCICE PRATIQUE 2.4Regroupez les gares par categorie_gare et calculez :- Le nombre de gares par categorie- La latitude moyenne- La longitude moyenne- Triez par nombre de gares decroissant

In [ ]:

# A COMPLETER
from pyspark.sql.functions import count, avg, col

df_par_categorie = df_gares.groupBy("categorie_gare").agg(
    count("*").alias("nombre_gares"),
    avg("latitude").alias("latitude_moyenne"),
    avg("longitude").alias("longitude_moyenne")
).orderBy(col("nombre_gares").desc())

df_par_categorie.show()


## PARTIE 4 : Agregations multiples

In [ ]:

print("EXERCICE 2.5 : Multiples agregations par groupe")
print("=" * 70)

from pyspark.sql.functions import count, avg, min, max, sum, when, col

# Agregations complexes par departement
df_stats_dept = df_gares.groupBy("code_departement").agg(
    count("*").alias("total_gares"),
    count(when(col("segment_clean") == "A", 1)).alias("gares_A"),
    count(when(col("segment_clean") == "B", 1)).alias("gares_B"),
    count(when(col("segment_clean") == "C", 1)).alias("gares_C"),
    avg("latitude").alias("lat_moyenne"),
    min("latitude").alias("lat_min"),
    max("latitude").alias("lat_max")
).orderBy(col("total_gares").desc())

print("Statistiques detaillees par departement :")
df_stats_dept.show(15)


## EXERCICE PRATIQUE 2.6Pour chaque code_departement, calculez :- Le nombre total de gares- Le pourcentage de gares principales (segment A)- La gare la plus au nord (max latitude)- La gare la plus au sud (min latitude)Affichez uniquement les departements avec plus de 10 gares

In [ ]:

# A COMPLETER
from pyspark.sql.functions import count, sum, when, col, min, max, round

df_stats_avancees = df_gares.groupBy("code_departement").agg(
    count("*").alias("total_gares"),
    (count(when(col("segment_clean") == "A", 1)) * 100.0 / count("*")).alias("pourcentage_A"),
    max("latitude").alias("gare_plus_nord"),
    min("latitude").alias("gare_plus_sud")
).filter(col("total_gares") > 10) \
 .orderBy(col("total_gares").desc())

df_stats_avancees.show()


## PARTIE 5 : Agregations avec HAVING (filtre apres groupBy)

In [ ]:

print("EXERCICE 2.7 : Filtrer apres agregation")
print("=" * 70)

from pyspark.sql.functions import count, col

# Trouver les departements avec plus de 20 gares
df_depts_importants = df_gares.groupBy("code_departement").agg(
    count("*").alias("nombre_gares")
).filter(col("nombre_gares") > 20) \
 .orderBy(col("nombre_gares").desc())

print("Departements avec plus de 20 gares :")
df_depts_importants.show()


## PARTIE 6 : Jointures

In [ ]:

print("EXERCICE 2.8 : Jointures entre tables")
print("=" * 70)

from pyspark.sql.functions import col

# Creer une table de reference des departements
data_depts = [
    ("75", "Paris", "Ile-de-France"),
    ("69", "Rhone", "Auvergne-Rhone-Alpes"),
    ("13", "Bouches-du-Rhone", "Provence-Alpes-Cote d'Azur"),
    ("59", "Nord", "Hauts-de-France"),
    ("33", "Gironde", "Nouvelle-Aquitaine"),
    ("44", "Loire-Atlantique", "Pays de la Loire"),
    ("67", "Bas-Rhin", "Grand Est"),
    ("31", "Haute-Garonne", "Occitanie"),
    ("06", "Alpes-Maritimes", "Provence-Alpes-Cote d'Azur"),
    ("92", "Hauts-de-Seine", "Ile-de-France")
]

df_depts_ref = spark.createDataFrame(
    data_depts,
    ["code_dept", "nom_dept", "region"]
)

# Sauvegarder la table de reference
df_depts_ref.write.format("delta").mode("overwrite").saveAsTable("departements_ref")

print("Table de reference creee")
df_depts_ref.show()

# Jointure INNER (garde uniquement les correspondances)
df_join_inner = df_gares.join(
    df_depts_ref,
    df_gares.code_departement == df_depts_ref.code_dept,
    "inner"
).select(
    "nom",
    "trigramme",
    "nom_dept",
    "region",
    "segment_clean"
)

print(f"\nJointure INNER : {df_join_inner.count()} lignes")
df_join_inner.show(10)

# Jointure LEFT (garde toutes les gares, meme sans correspondance)
df_join_left = df_gares.join(
    df_depts_ref,
    df_gares.code_departement == df_depts_ref.code_dept,
    "left"
).select(
    "nom",
    "code_departement",
    "nom_dept",
    "region"
)

print(f"\nJointure LEFT : {df_join_left.count()} lignes")
df_join_left.filter(col("nom_dept").isNull()).show(10)


## EXERCICE PRATIQUE 2.9Faites une jointure entre gares_silver et departements_ref, puis :1. Comptez le nombre de gares par region2. Calculez le nombre de gares principales (A) par region3. Triez par nombre de gares decroissant

In [ ]:

# A COMPLETER
from pyspark.sql.functions import count, when, col

# Jointure
df_gares_region = df_gares.join(
    df_depts_ref,
    df_gares.code_departement == df_depts_ref.code_dept,
    "inner"
)

# Agregation par region
df_stats_region = df_gares_region.groupBy("region").agg(
    count("*").alias("total_gares"),
    count(when(col("segment_clean") == "A", 1)).alias("gares_principales")
).orderBy(col("total_gares").desc())

print("Statistiques par region :")
df_stats_region.show()


## PARTIE 7 : Fonctions de fenetre (apercu)

In [ ]:

print("EXERCICE 2.10 : Introduction aux Window Functions")
print("=" * 70)

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, col

# Creer une fenetre partitionnee par departement, triee par nom
window_spec = Window.partitionBy("code_departement").orderBy("nom")

# Ajouter un numero de ligne par departement
df_avec_rang = df_gares \
    .filter(col("code_departement").isin("75", "69", "13")) \
    .withColumn("rang_dans_dept", row_number().over(window_spec)) \
    .select("code_departement", "nom", "trigramme", "rang_dans_dept")

print("Rang de chaque gare dans son departement :")
df_avec_rang.show(20)

# Trouver la premiere gare de chaque departement (ordre alphabetique)
df_premieres = df_avec_rang.filter(col("rang_dans_dept") == 1)
print("\nPremiere gare de chaque departement (alphabetiquement) :")
df_premieres.show()


## PARTIE 8 : Pivot tables

In [ ]:

print("EXERCICE 2.11 : Tables croisees dynamiques")
print("=" * 70)

from pyspark.sql.functions import count, col

# Creer un pivot : segments en lignes, departements en colonnes
df_pivot = df_gares \
    .filter(col("code_departement").isin("75", "69", "13", "59", "33")) \
    .groupBy("segment_clean") \
    .pivot("code_departement") \
    .agg(count("*"))

print("Pivot : nombre de gares par segment et departement :")
df_pivot.show()

# Pivot inverse : departements en lignes, segments en colonnes
df_pivot_inverse = df_gares \
    .filter(col("code_departement").isin("75", "69", "13", "59", "33")) \
    .groupBy("code_departement") \
    .pivot("segment_clean") \
    .agg(count("*"))

print("\nPivot inverse :")
df_pivot_inverse.show()


## EXERCICE FINAL 2.12 : Projet complet d'analyseCreez une analyse complete qui :1. Joint les gares avec la table de reference des departements2. Groupe par region et segment3. Calcule pour chaque groupe :- Le nombre de gares- La latitude moyenne- La longitude moyenne4. Filtre uniquement les groupes avec plus de 5 gares5. Trie par region puis par nombre de gares decroissant6. Sauvegarde le resultat dans une nouvelle table

In [ ]:

# A COMPLETER - SOLUTION PROPOSEE
from pyspark.sql.functions import count, avg, col

# Pipeline complete
df_analyse_complete = df_gares \
    .join(df_depts_ref, df_gares.code_departement == df_depts_ref.code_dept, "inner") \
    .groupBy("region", "segment_clean") \
    .agg(
        count("*").alias("nombre_gares"),
        avg("latitude").alias("latitude_moyenne"),
        avg("longitude").alias("longitude_moyenne")
    ) \
    .filter(col("nombre_gares") > 5) \
    .orderBy("region", col("nombre_gares").desc())

print("Analyse complete :")
df_analyse_complete.show(30)

# Sauvegarder
df_analyse_complete.write.format("delta").mode("overwrite").saveAsTable("analyse_gares_region")
print("\nTable 'analyse_gares_region' creee !")


## PARTIE 9 : Agregations avec SQL

In [ ]:

%sql
-- EXERCICE 2.13 : Meme analyse en SQL

SELECT
segment_clean,
COUNT(*) as nombre_gares,
ROUND(AVG(latitude), 4) as lat_moyenne,
ROUND(AVG(longitude), 4) as lon_moyenne,
MIN(latitude) as lat_min,
MAX(latitude) as lat_max
FROM gares_silver
GROUP BY segment_clean
ORDER BY segment_clean


In [ ]:

%sql
-- Top 10 departements
SELECT
code_departement,
COUNT(*) as total,
SUM(CASE WHEN segment_clean = 'A' THEN 1 ELSE 0 END) as gares_A,
SUM(CASE WHEN segment_clean = 'B' THEN 1 ELSE 0 END) as gares_B,
SUM(CASE WHEN segment_clean = 'C' THEN 1 ELSE 0 END) as gares_C
FROM gares_silver
GROUP BY code_departement
ORDER BY total DESC
LIMIT 10


## RESUME DES CONCEPTS APPRISDans cet exercice, vous avez appris :1. **Agregations simples** : `count()`, `sum()`, `avg()`, `min()`, `max()`2. **Regroupements** : `groupBy()`, `agg()`3. **Agregations conditionnelles** : `count(when())`4. **Filtres post-agregation** : `filter()` apres `groupBy()`5. **Jointures** : `join()` avec INNER, LEFT, RIGHT6. **Fonctions de fenetre** : `Window`, `row_number()`, `rank()`7. **Pivot tables** : `pivot()`8. **SQL avance** : `GROUP BY`, `HAVING`, `CASE WHEN`## Prochaine etapePassez a l'**Exercice 03 : Fonctions de fenetre avancees** pour maitriser :- Les calculs cumules (running totals)- Les classements (ranking)- Les decalages (lag/lead)- Les moyennes mobiles